In [ ]:
#!pip install -q accelerate transformers einops datasets peft bitsandbytes --upgrade
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets einops trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


#<font color = lightblue> 모델 불러오기 및 설정

In [ ]:
from peft import PeftModel
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import os
from accelerate import Accelerator, FullyShardedDataParallelPlugin
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

In [ ]:
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("daekeun-ml/phi-2-ko-v0.1", trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/724k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.04M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "daekeun-ml/phi-2-ko-v0.1",
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

#model = AutoModelForCausalLM.from_pretrained("daekeun-ml/phi-2-ko-v0.1", torch_dtype="auto", quantization_config=bnb_config)

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/daekeun-ml/phi-2-ko-v0.1:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/daekeun-ml/phi-2-ko-v0.1:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/761M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(66676, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [ ]:
from bitsandbytes.nn.modules import Linear8bitLt, Linear4bit
import torch.nn.init as init  # Importing the init module from PyTorch
import bitsandbytes as bnb
from contextlib import contextmanager

def noop (x=None, *args, **kwargs):
    "Do nothing"
    return x

@contextmanager
def no_kaiming():
    old_iku = init.kaiming_uniform_
    init.kaiming_uniform_ = noop
    try: yield
    finally: init.kaiming_uniform_ = old_iku

_old_8init = Linear8bitLt.__init__
_old_4init = Linear4bit.__init__

def _new_4init(self, input_features, output_features, bias=True,
               device=None, **kwargs):
    with no_kaiming():
        return _old_4init(self, input_features, output_features, bias=bias,
                          device=device, **kwargs)



def _new_8init(self, input_features, output_features, bias=True, has_fp16_weights=True,
              memory_efficient_backward=False, threshold=0.0, index=None, device=None):
    with no_kaiming():
        return _old_8init(self, input_features, output_features, bias=bias, has_fp16_weights=has_fp16_weights,
                          memory_efficient_backward=memory_efficient_backward, threshold=threshold, index=index, device=device)

Linear8bitLt.__init__ = _new_8init
Linear4bit.__init__ = _new_4init

In [ ]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    #target_modules=["Wqkv","fc1","fc2"],
    target_modules=['q_proj', 'v_proj', 'fc1', 'fc2'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 36,700,160 || all params: 2,895,636,596 || trainable%: 1.2674297614105716


In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(66676, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(i

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(66676, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(i

#<font color = lightblue> 학습데이터 전처리

In [ ]:
def tokenize(sample):
    tokenized_text =  tokenizer(sample["text"], padding=True, truncation=True, max_length=150)    # truncation = 지정된 길이보다 긴 문장은 자르기
    return tokenized_text

In [ ]:
dataset = load_dataset('json', data_files='wellness_dialog_combined_train.jsonl', split='train')
dataset_df = dataset.to_pandas()
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['input', 'output'],
    num_rows: 4265
})

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset)
df.head(2)

,input,output
0,내가 놀라고 싶지 않다 해서 그럴 수 있는 것도 아니고…,그런 일이 있으셨군요.
1,대학 떨어질 줄은 정말 상상도 못했어요.,상심이 크셨겠어요.


In [ ]:
dataset_df["text"] = dataset_df[['input', 'output']].apply(lambda x: "input: " + x["input"] + "output: " + x['output'], axis=1)
dataset_df.head(2)

,input,output,text
0,내가 놀라고 싶지 않다 해서 그럴 수 있는 것도 아니고…,그런 일이 있으셨군요.,input: 내가 놀라고 싶지 않다 해서 그럴 수 있는 것도 아니고…output: ...
1,대학 떨어질 줄은 정말 상상도 못했어요.,상심이 크셨겠어요.,input: 대학 떨어질 줄은 정말 상상도 못했어요.output: 상심이 크셨겠어요.


In [ ]:
dataset = Dataset.from_pandas(dataset_df)
tokenized_data = dataset.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=dataset.column_names)

tokenized_data

Tokenizing data:   0%|          | 0/4265 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4265
})

In [ ]:
eval_dataset = load_dataset('json', data_files='wellness_dialog_combined_validation_proper.jsonl', split='train')
eval_dataset_df = eval_dataset.to_pandas()
eval_dataset_df

,input,output
0,옷을 입고 지내는 게 너무 싫어요.,"아, 그러시군요. 꺼끄러운 면이 닿으면 불편하죠."
1,안 먹어야지 해도 회사에서 스트레스 받으면 어떻게 할 수가 없어요.,직장 스트레스가 많으시군요. 그럴 땐 정말 멀리 도망가버리고 싶죠.
2,"과CC였는데, 좀 안 좋게 헤어졌어.",마음이 아프네요. 아름다운 사랑만큼 아름다운 이별은 왜 없는 걸까요?
3,그냥 들판에 있었는데 뭐가 자꾸 날아다니는 거야.,그러시군요. 그 다음은 어떻게 됐나요?
4,몸무게 유지하려고 거기서는 과일만 먹었어요.,정말 멋져요. 다이어트 정말 힘들잖아요.
...,...,...
468,사람 만나기가 싫어…,충분히 그럴 수 있죠. 너무 불편하면 굳이 애써서 만날 필요도 없답니다.
469,여자친구랑 떨어지기가 싫었어.,"아, 그렇군요."
470,뒷머리가 무겁고 뒷목이 뻣뻣해지고 어깨가 걸리더라구.,적당한 양의 진통제가 도움이 될 수 있어요.
471,집중력도 남들 반도 안되는것 같아요.,집중이 잘 안 될 때가 있죠. 그럴 때는 아예 잠깐 다른 일을 하는 것도 좋아요.


In [ ]:
eval_dataset_df["Text"] = eval_dataset_df[['input', 'output']].apply(lambda x: "[INST]" + x["input"] + "[/INST]" + x['output'], axis=1)
eval_dataset_df.head(2)

,input,output,Text
0,옷을 입고 지내는 게 너무 싫어요.,"아, 그러시군요. 꺼끄러운 면이 닿으면 불편하죠.","[INST]옷을 입고 지내는 게 너무 싫어요.[/INST]아, 그러시군요. 꺼끄러운..."
1,안 먹어야지 해도 회사에서 스트레스 받으면 어떻게 할 수가 없어요.,직장 스트레스가 많으시군요. 그럴 땐 정말 멀리 도망가버리고 싶죠.,[INST]안 먹어야지 해도 회사에서 스트레스 받으면 어떻게 할 수가 없어요.[/I...


In [ ]:
eval_dataset = eval_dataset_df['Text']
eval_dataset

0      [INST]옷을 입고 지내는 게 너무 싫어요.[/INST]아, 그러시군요. 꺼끄러운...
1      [INST]안 먹어야지 해도 회사에서 스트레스 받으면 어떻게 할 수가 없어요.[/I...
2      [INST]과CC였는데, 좀 안 좋게 헤어졌어.[/INST]마음이 아프네요. 아름다...
3      [INST]그냥 들판에 있었는데 뭐가 자꾸 날아다니는 거야.[/INST]그러시군요....
4      [INST]몸무게 유지하려고 거기서는 과일만 먹었어요.[/INST]정말 멋져요. 다...
                             ...                        
468    [INST]사람 만나기가 싫어…[/INST]충분히 그럴 수 있죠. 너무 불편하면 굳...
469                [INST]여자친구랑 떨어지기가 싫었어.[/INST]아, 그렇군요.
470    [INST]뒷머리가 무겁고 뒷목이 뻣뻣해지고 어깨가 걸리더라구.[/INST]적당한 ...
471    [INST]집중력도 남들 반도 안되는것 같아요.[/INST]집중이 잘 안 될 때가 ...
472    [INST]또 크게 어지럼증이 느껴져서 쓰러지기라도 하면 어떡하나, 큰일이 나면 어...
Name: Text, Length: 473, dtype: object

In [ ]:
new_df = eval_dataset_df[['Text']]
new_df.head(2)

,Text
0,"[INST]옷을 입고 지내는 게 너무 싫어요.[/INST]아, 그러시군요. 꺼끄러운..."
1,[INST]안 먹어야지 해도 회사에서 스트레스 받으면 어떻게 할 수가 없어요.[/I...


In [ ]:
new_df.to_csv('wellness_dialog_eval.csv', index=False)    # csv 저장코드

#<font color = lightblue> 미리 처리해둔 데이터 가져오기

In [ ]:
training_dataset = load_dataset("csv", data_files="wellness_dialog_train.csv", split='train')
training_dataset

Dataset({
    features: ['Text'],
    num_rows: 4265
})

In [ ]:
eval_dataset = load_dataset("csv", data_files="wellness_dialog_eval.csv", split='train')
eval_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['Text'],
    num_rows: 473
})

#<font color=lightblue> fine tune

In [ ]:
!pip list

In [ ]:
from datetime import datetime
from trl import SFTTrainer

In [ ]:
project = "finall-project"
base_model_name = "phi2-v3"
run_name = base_model_name + "-" + project
#output_dir = "./" + run_name
output_dir = run_name

'''
training_arguments = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=1000,
        num_train_epochs=1
    )
'''
training_arguments = TrainingArguments(
    output_dir=output_dir,
    warmup_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    max_steps=1000,
    #learning_rate=2.5e-5, # Want a small lr for finetuning
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_steps=25,              # When to start reporting loss
    logging_dir="./logs",        # Directory for storing logs
    save_strategy="steps",       # Save the model checkpoint every logging step
    save_steps=25,                # Save checkpoints every 50 steps
    evaluation_strategy="steps", # Evaluate the model every logging step
    do_eval=True,                # Perform evaluation at the end of training
    #report_to="wandb",           # Comment this out if you don't want to use weights & baises
    run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
)

peft_config = LoraConfig(
    r = 32,
    lora_alpha = 64,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules = ["Wqkv", "fc1", "fc2"]
)

trainer = SFTTrainer(
    model = model,
    train_dataset = training_dataset,
    eval_dataset = eval_dataset,
    peft_config = peft_config,
    dataset_text_field = "Text",
    max_seq_length = 690,
    tokenizer = tokenizer,
    args = training_arguments
)

Map:   0%|          | 0/4265 [00:00<?, ? examples/s]

Map:   0%|          | 0/473 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,4.480300,4.900298
50,4.424400,4.900298
75,4.408900,4.900298
100,4.461400,4.900298
125,4.415600,4.900298
150,4.421400,4.900298
175,4.479800,4.900298
200,4.521900,4.900298
225,4.422500,4.900298
250,4.449500,4.900298


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=1000, training_loss=4.460945167541504, metrics={'train_runtime': 5317.5538, 'train_samples_per_second': 1.504, 'train_steps_per_second': 0.188, 'total_flos': 5528365067944728.0, 'train_loss': 4.460945167541504, 'epoch': 1.87})

#<font color = lightblue> HuggingFace 업로드

In [ ]:
from huggingface_hub import notebook_login

notebook_login()    

In [ ]:
model.push_to_hub("ledu1017/phi2-ko-mental-chatbot",
                  #use_auth_token=True,
                  token = '',
                  commit_message="basic training",
                  private=True)

adapter_model.safetensors:   0%|          | 0.00/147M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ledu1017/phi2-ko-mental-chatbot/commit/33b20997c54330caf88d49b1c1c2ba52a9b342c9', commit_message='basic training', commit_description='', oid='33b20997c54330caf88d49b1c1c2ba52a9b342c9', pr_url=None, pr_revision=None, pr_num=None)

#<font color=lightblue> 모델 테스트

In [ ]:
from transformers import AutoTokenizer
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

# 모델과 토크나이저 초기화

tokenizer = AutoTokenizer.from_pretrained("daekeun-ml/phi-2-ko-v0.1")
base_model = AutoModelForCausalLM.from_pretrained("daekeun-ml/phi-2-ko-v0.1")
config = PeftConfig.from_pretrained("ledu1017/phi2-ko-mental-chat")
model = PeftModel.from_pretrained(base_model, "ledu1017/phi2-ko-mental-chat")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/105M [00:00<?, ?B/s]

In [ ]:
def generate_response(user_input):
    # 사용자 입력을 토크나이즈
    input_ids = tokenizer.encode(user_input, return_tensors='pt')

    # 모델을 사용하여 응답 생성
    output = model.generate(input_ids, max_length=50, num_return_sequences=1)

    # 생성된 토큰들을 문자열로 변환
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# 사용자 입력
user_input = "[INST]안녕?[/INST]"

# 생성된 답변 출력
response = generate_response(user_input)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[INST]안녕?[/INST]그런 일이 있으셨군요. 기분이 많이 나빴겠어요. 지금은 괜찮으신가요?[/INST]괜찮아요. 저도 자주 깜빡하는 걸요 하지만 깜빡하는 건 나쁜 일이 아니
